In [1]:
import pandas as pd
import numpy as np
# from sklearn.naive_bayes import GaussianNB
# from sklearn import svm
# from sklearn import tree
# from sklearn.neighbors import KNeighborsClassifier
# from sklearn.linear_model import LogisticRegression
# from sklearn.metrics import accuracy_score
# from sklearn.cross_validation import train_test_split
import tensorflow as tf

In [2]:
data = pd.read_csv("Prototype_data_DL.csv", sep = ",")

In [3]:
data.head()

,File,label,1,2,3,4,5,6,7,8
0,1,0,0,0,0,0,0,2,1,26
1,2,0,0,0,0,0,2,1,16,34
2,3,1,0,0,6,2,1,4,7,8
3,4,1,31,32,33,36,2,1,1,26
4,5,0,36,2,1,2,1,2,1,8


In [4]:
data.shape

(10, 10)

In [5]:
l1 = [0,0,0,0,0,2,1,26]
l2 = [0,0,0,0,2,1,16,34]
l3 = [0,0,6,2,1,4,7,8]
l4 = [31,32,33,36,2,1,1,26]
l5 = [36,2,1,2,1,2,1,8]
l6 = [0,0,0,0,0,0,2,1]
l7 = [0,0,0,0,2,1,16,34]
l8 = [0,36,2,1,1,4,7,8]
l9 = [31,2,1,2,2,1,1,26]
l10 = [36,2,1,2,1,2,1,8]
sequences = []
sequences.append(l1)
sequences.append(l2)
sequences.append(l3)
sequences.append(l4)
sequences.append(l5)
sequences.append(l6)
sequences.append(l7)
sequences.append(l8)
sequences.append(l9)
sequences.append(l10)

In [6]:
sequences

[[0, 0, 0, 0, 0, 2, 1, 26],
 [0, 0, 0, 0, 2, 1, 16, 34],
 [0, 0, 6, 2, 1, 4, 7, 8],
 [31, 32, 33, 36, 2, 1, 1, 26],
 [36, 2, 1, 2, 1, 2, 1, 8],
 [0, 0, 0, 0, 0, 0, 2, 1],
 [0, 0, 0, 0, 2, 1, 16, 34],
 [0, 36, 2, 1, 1, 4, 7, 8],
 [31, 2, 1, 2, 2, 1, 1, 26],
 [36, 2, 1, 2, 1, 2, 1, 8]]

In [7]:
X_train = sequences[:5]
y_train = data.iloc[:5, 1].values
X_test = sequences[5:]
y_test = data.iloc[5:, 1].values

In [8]:
X_test

[[0, 0, 0, 0, 0, 0, 2, 1],
 [0, 0, 0, 0, 2, 1, 16, 34],
 [0, 36, 2, 1, 1, 4, 7, 8],
 [31, 2, 1, 2, 2, 1, 1, 26],
 [36, 2, 1, 2, 1, 2, 1, 8]]

In [9]:
def create_batch_generator(x, y=None, batch_size=2):
    n_batches = len(x)//batch_size
    x= x[:n_batches*batch_size]
    if y is not None:
        y = y[:n_batches*batch_size]
    for ii in range(0, len(x), batch_size):
        if y is not None:
            yield x[ii:ii+batch_size], y[ii:ii+batch_size]
        else:
            yield x[ii:ii+batch_size]

In [10]:
class AnalyzeLogRNN(object):
    def __init__(self, n_words, seq_len=8,
                 lstm_size=256, num_layers=2, batch_size=5,
                 learning_rate=0.0001, embed_size=8):
        self.n_words = n_words
        self.seq_len = seq_len
        self.lstm_size = lstm_size 
        self.num_layers = num_layers
        self.batch_size = batch_size
        self.learning_rate = learning_rate
        self.embed_size = embed_size

        self.g = tf.Graph()
        with self.g.as_default():
            tf.set_random_seed(123)
            self.build()
            self.saver = tf.train.Saver()
            self.init_op = tf.global_variables_initializer()

    def build(self):
        tf_x = tf.placeholder(tf.int32,
                              shape=(self.batch_size, self.seq_len),
                              name='tf_x')
        tf_y = tf.placeholder(tf.float32,
                              shape=(self.batch_size),
                              name='tf_y')
        tf_keepprob = tf.placeholder(tf.float32,
                                     name='tf_keepprob')
        
        embedding = tf.Variable(
            tf.random_uniform(
                (self.n_words, self.embed_size),
                minval=-1, maxval=1),
            name='embedding')
        embed_x = tf.nn.embedding_lookup(
            embedding, tf_x,
            name='embeded_x')

        cells = tf.contrib.rnn.MultiRNNCell(
            [tf.contrib.rnn.DropoutWrapper(
                tf.contrib.rnn.BasicLSTMCell(self.lstm_size),
                output_keep_prob=tf_keepprob)
                for i in range(self.num_layers)])

        self.initial_state = cells.zero_state(
            self.batch_size, tf.float32)
        print('  << initial state >> ', self.initial_state)

        lstm_outputs, self.final_state = tf.nn.dynamic_rnn(
            cells, embed_x,
            initial_state=self.initial_state)
        
        print('\n  << lstm_output   >> ', lstm_outputs)
        print('\n  << final state   >> ', self.final_state)

        logits = tf.layers.dense(
            inputs=lstm_outputs[:, -1],
            units=1, activation=None,
            name='logits')

        logits = tf.squeeze(logits, name='logits_squeezed')
        print ('\n  << logits        >> ', logits)

        y_proba = tf.nn.sigmoid(logits, name='probabilities')
        predictions = {
            'probabilities': y_proba,
            'labels' : tf.cast(tf.round(y_proba), tf.int32,
                               name='labels')
        }
        print('\n  << predictions   >> ', predictions)

        cost = tf.reduce_mean(
            tf.nn.sigmoid_cross_entropy_with_logits(
                labels=tf_y, logits=logits),
            name='cost')

        optimizer = tf.train.AdamOptimizer(self.learning_rate)
        train_op = optimizer.minimize(cost, name='train_op')

    def train(self, X_train, y_train, num_epochs):
        with tf.Session(graph=self.g) as sess:
            sess.run(self.init_op)
            iteration = 1
            for epoch in range(num_epochs):
                state = sess.run(self.initial_state)

                for batch_x, batch_y in create_batch_generator(
                        X_train, y_train, self.batch_size):
                    feed = {'tf_x:0': batch_x,
                            'tf_y:0': batch_y,
                            'tf_keepprob:0': 0.5,
                            self.initial_state : state}
                    loss, _, state = sess.run(
                        ['cost:0', 'train_op',
                         self.final_state],
                        feed_dict=feed)

                    if iteration % 20 == 0:
                        print("Epoch: %d/%d Iteration: %d "
                              "| Train loss: %.5f" % (
                                  epoch + 1, num_epochs,
                                  iteration, loss))

                    iteration +=1
                if (epoch+1)%10 == 0:
                    self.saver.save(sess,
                                    "model/sentiment-%d.ckpt" % epoch)

    def predict(self, X_data, return_proba=False):
        preds = []
        with tf.Session(graph = self.g) as sess:
            self.saver.restore(
                sess, tf.train.latest_checkpoint('model/'))
            test_state = sess.run(self.initial_state)
            for ii, batch_x in enumerate(
                    create_batch_generator(
                        X_data, None, batch_size=self.batch_size), 1):
                feed = {'tf_x:0' : batch_x,
                        'tf_keepprob:0': 1.0,
                        self.initial_state : test_state}
                if return_proba:
                    pred, test_state = sess.run(
                        ['probabilities:0', self.final_state],
                        feed_dict=feed)
                else:
                    pred, test_state = sess.run(
                        ['labels:0', self.final_state],
                        feed_dict=feed)

                preds.append(pred)

        return np.concatenate(preds)


In [11]:
n_words = 42
sequence_length = 8

In [12]:
rnn = AnalyzeLogRNN(n_words=n_words,
                   seq_len=sequence_length,
                   embed_size=8,
                   lstm_size=256,
                   num_layers=2,
                   batch_size=5,
                   learning_rate=0.001)

  << initial state >>  (LSTMStateTuple(c=<tf.Tensor 'MultiRNNCellZeroState/DropoutWrapperZeroState/BasicLSTMCellZeroState/zeros:0' shape=(5, 256) dtype=float32>, h=<tf.Tensor 'MultiRNNCellZeroState/DropoutWrapperZeroState/BasicLSTMCellZeroState/zeros_1:0' shape=(5, 256) dtype=float32>), LSTMStateTuple(c=<tf.Tensor 'MultiRNNCellZeroState/DropoutWrapperZeroState_1/BasicLSTMCellZeroState/zeros:0' shape=(5, 256) dtype=float32>, h=<tf.Tensor 'MultiRNNCellZeroState/DropoutWrapperZeroState_1/BasicLSTMCellZeroState/zeros_1:0' shape=(5, 256) dtype=float32>))

  << lstm_output   >>  Tensor("rnn/transpose_1:0", shape=(5, 8, 256), dtype=float32)

  << final state   >>  (LSTMStateTuple(c=<tf.Tensor 'rnn/while/Exit_3:0' shape=(5, 256) dtype=float32>, h=<tf.Tensor 'rnn/while/Exit_4:0' shape=(5, 256) dtype=float32>), LSTMStateTuple(c=<tf.Tensor 'rnn/while/Exit_5:0' shape=(5, 256) dtype=float32>, h=<tf.Tensor 'rnn/while/Exit_6:0' shape=(5, 256) dtype=float32>))

  << logits        >>  Tensor("logits_sq

In [13]:
rnn.train(X_train, y_train, num_epochs=5)

In [14]:
proba = rnn.predict(X_test, return_proba=True)

INFO:tensorflow:Restoring parameters from model/sentiment-9.ckpt


In [15]:
proba

array([  9.61865808e-05,   4.46955266e-04,   9.54227805e-01,
         9.91116881e-01,   9.91954267e-01], dtype=float32)

In [17]:
pred = rnn.predict(X_test)
pred #predicted by RNN LSTM model

INFO:tensorflow:Restoring parameters from model/sentiment-9.ckpt


array([0, 0, 1, 1, 1], dtype=int32)

In [18]:
y_test # Actual label

array([0, 0, 1, 0, 1])

In [ ]:
#label: 1 - success, 0 - failure
#So the test data had three failures and two success logs
#RNN LSTM model predicted both successes correctly, it predicted two out of three failures correctly and 
# misclassified one as success